# Scale Data with Outliers

Adapted from Jason Brownlee. 2020. [Data Preparation for Machine Learning](https://machinelearningmastery.com/data-preparation-for-machine-learning/).


## Overview

Many machine learning algorithms perform better when numerical input variables are scaled to
a standard range. This includes algorithms that use a weighted sum of the input, like linear
regression, and algorithms that use distance measures, like k-nearest neighbors. Standardizing
is a popular scaling technique that subtracts the mean from values and divides by the standard
deviation, transforming the probability distribution for an input variable to a standard Gaussian
(zero mean and unit variance). Standardization can become skewed or biased if the input
variable contains outlier values.
To overcome this, the median and interquartile range can be used when standardizing
numerical input variables, generally referred to as robust scaling.

## Learning Objectives

- Understand why many machine learning algorithms prefer scaled numerical input variables
- Learn robust scaling techniques that use percentiles to scale numerical input variables containing outliers
- Master using `RobustScaler` to scale numerical input variables using median and interquartile range
- Evaluate the impact of different scaling ranges on model performance

### Tasks to complete

- Examine raw dataset distributions
- Apply robust scaling transformation
- Evaluate model performance with different scaling ranges
- Compare results between scaled and unscaled data


## Prerequisites

- A working Python environment
- Basic understanding of Python programming concepts
- Basic understanding of machine learning concepts
- Familiarity with pandas and numpy libraries
- Knowledge of basic statistical concepts


## Get Started

To start, we install required packages and import necessary libraries.


### Install packages

In [ ]:
# Install necessary Python libraries using pip
%pip install matplotlib numpy pandas scikit-learn  

# matplotlib - A library for creating static, animated, and interactive visualizations in Python.
# numpy - A fundamental package for numerical computing, providing support for arrays and mathematical operations.
# pandas - A powerful data analysis and manipulation library, useful for working with structured data.
# scikit-learn - A machine learning library that provides simple and efficient tools for data mining and analysis.


### Import libraries

In [ ]:
# Import necessary libraries
from matplotlib import pyplot  # For plotting graphs
from numpy import mean, std  # For calculating mean and standard deviation
from pandas import DataFrame, read_csv  # For handling data as DataFrames and reading CSV files
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score  # For cross-validation
from sklearn.neighbors import KNeighborsClassifier  # K-Nearest Neighbors classifier
from sklearn.pipeline import Pipeline  # To create machine learning pipelines
from sklearn.preprocessing import LabelEncoder, RobustScaler  # For data preprocessing

# Define the path to the dataset
pima_indians_diabetes_csv = "../../Data/pima-indians-diabetes.csv"  

## Robust Scaling Data

If there are input
variables that have very large values relative to the other input variables, these large values
can dominate or skew some machine learning algorithms. The result is that the algorithms pay
most of their attention to the large values and ignore the variables with smaller values.

Sometimes an input variable may have outlier values. Outliers can skew a probability distribution and make data scaling using standardization
difficult as the calculated mean and standard deviation will be skewed by the presence of
the outliers. 

Robust standardization or robust data
scaling that calculates the median (50th percentile) and the 25th and
75th percentiles. The values of each variable then have their median subtracted and are divided
by the interquartile range (IQR) which is the difference between the 75th and 25th percentiles


## Diabetes Dataset

The dataset classifies patient data as
either an onset of diabetes within five years or not.

```
Number of Instances: 768
Number of Attributes: 8 plus class 
For Each Attribute: (all numeric-valued)
   1. Number of times pregnant
   2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
   3. Diastolic blood pressure (mm Hg)
   4. Triceps skin fold thickness (mm)
   5. 2-Hour serum insulin (mu U/ml)
   6. Body mass index (weight in kg/(height in m)^2)
   7. Diabetes pedigree function
   8. Age (years)
   9. Class variable (0 or 1)
Missing Attribute Values: Yes
Class Distribution: (class value 1 is interpreted as "tested positive for
   diabetes")
   Class Value  Number of instances
   0            500
   1            268
```

You can learn more about the dataset here:

* Diabetes Dataset File ([pima-indians-diabetes.csv](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv))
* Diabetes Dataset Details ([pima-indians-diabetes.names](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.names))

In [ ]:
# Load and summarize the diabetes dataset

# Define the dataset file path (Ensure 'pima_indians_diabetes_csv' is correctly defined)
dataset = read_csv(pima_indians_diabetes_csv, header=None)  # Load dataset without headers
print(dataset.head())  # Display the first five rows of the dataset

# Summarize the shape of the dataset
print(dataset.shape)  # Print the number of rows and columns in the dataset

# Summarize each variable (statistical summary)
print(dataset.describe())  # Display summary statistics (count, mean, std, min, max, etc.)

This confirms the 8
input variables, one output variable, and 768 rows of data. A statistical summary of the input
variables is provided show that each variable has a very different scale. This makes it a good
dataset for exploring data scaling methods.

We can create a histogram for each input variable. The plots confirm the differing scale
for each input variable and show that the variables have differing scales. Importantly, we can see
some of the distributions show the presence of outliers. The dataset provides a good candidate for using a robust scaler transform to standardize the data in the presence of di ering input
variable scales and outliers.

In [ ]:
# Generate histograms for all variables in the dataset
# xlabelsize and ylabelsize control the font size of axis labels
fig = dataset.hist(xlabelsize=4, ylabelsize=4)

# Reduce the title size of each subplot for better readability
[x.title.set_size(4) for x in fig.ravel()]

# Display the histogram plot
pyplot.show()

Next, let's fit and evaluate a machine learning model on the raw dataset. We will use
a k-nearest neighbor algorithm with default hyperparameters and evaluate it using repeated
stratified k-fold cross-validation.

In [ ]:
# Evaluate k-Nearest Neighbors (KNN) on the raw Pima Indians Diabetes dataset

# Load dataset from CSV file (Assuming 'pima_indians_diabetes_csv' is defined elsewhere)
dataset = read_csv(pima_indians_diabetes_csv, header=None)

# Convert the dataset into a NumPy array
data = dataset.values

# Separate the dataset into input features (X) and output labels (y)
X, y = data[:, :-1], data[:, -1]

# Ensure input features are of type float (for consistency in calculations)
X = X.astype("float32")

# Encode the output labels as integers (necessary for classification models)
y = LabelEncoder().fit_transform(y.astype("str"))

# Define the KNN model with default parameters
model = KNeighborsClassifier()

# Set up cross-validation with 10 splits, repeated 3 times for robust evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# Perform cross-validation and compute accuracy scores across different folds
n_scores = cross_val_score(model, X, y, scoring="accuracy", cv=cv, n_jobs=-1)

# Report the model's mean accuracy and standard deviation across all cross-validation runs
print("Accuracy: %.3f (%.3f)" % (mean(n_scores), std(n_scores)))

# Show any plots (though no plots are explicitly generated in this code)
pyplot.show()

In this case we can see that the model achieved a mean classi cationfiaccuracy of about
71.7 percent.

Next, let's explore a robust scaling transform of the dataset.

## IQR Robust Scaler Transform

We can apply the robust scaler to the diabetes dataset directly. We will use the default
configuration and scale values to the IQR. First, a **RobustScaler** instance is defined with
default hyperparameters. Once defined, we can call the **fit.transform()** function and pass it
to our dataset to create a robust scale transformed version of our dataset.

In [ ]:
# Load the Pima Indians Diabetes dataset from a CSV file
# `header=None` ensures that no row is treated as column names
dataset = read_csv(pima_indians_diabetes_csv, header=None)

# Extract only the numeric input features (excluding the target variable in the last column)
data = dataset.values[:, :-1]

# Initialize the RobustScaler, which scales features using statistics 
# that are robust to outliers (i.e., median and interquartile range)
trans = RobustScaler()

# Fit the scaler to the data and transform it to remove the influence of outliers
data = trans.fit_transform(data)

# Convert the transformed NumPy array back into a Pandas DataFrame for better readability
dataset = DataFrame(data)

# Display summary statistics of the transformed dataset
# This includes count, mean, standard deviation, min, and percentile values
print(dataset.describe())

We can see that the
distributions have been adjusted. The median values are now zero and the standard deviation
values are now close to 1.0.

In [ ]:
# Generate histograms for all variables in the dataset
# xlabelsize and ylabelsize adjust the font size of axis labels for readability
fig = dataset.hist(xlabelsize=4, ylabelsize=4)

# Reduce the title font size for each subplot in the figure
[x.title.set_size(4) for x in fig.ravel()]

# Display the histogram plot
pyplot.show()

Histogram plots of the variables are created, although the distributions don't look much
different from their original distributions seen in the previous section. We can see that the
center of mass for each distribution is now close to zero.

Next, let's evaluate the same KNN model as the previous section, but in this case on a
robust scaler transform of the dataset.

In [ ]:
# Evaluate K-Nearest Neighbors (KNN) on the Pima Indians Diabetes dataset 
# using RobustScaler for data preprocessing

# Load the dataset from CSV file
dataset = read_csv(pima_indians_diabetes_csv, header=None)  # Replace with actual file path
data = dataset.values  # Convert dataframe to NumPy array for processing

# Separate the dataset into input features (X) and target labels (y)
X, y = data[:, :-1], data[:, -1]

# Convert input features to float type and encode target labels as integers
X = X.astype("float32")  
y = LabelEncoder().fit_transform(y.astype("str"))  # Ensures target labels are numerically encoded

# Define a preprocessing and modeling pipeline
trans = RobustScaler()  # Use RobustScaler to handle outliers and normalize features
model = KNeighborsClassifier()  # Initialize KNN classifier
pipeline = Pipeline(steps=[("t", trans), ("m", model)])  # Create a pipeline with scaling and classification

# Define cross-validation strategy
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)  
# This splits data into 10 folds and repeats the process 3 times to ensure robust evaluation

# Evaluate the pipeline using cross-validation and compute accuracy scores
n_scores = cross_val_score(pipeline, X, y, scoring="accuracy", cv=cv, n_jobs=-1)

# Report mean accuracy and standard deviation of scores
print("Accuracy: %.3f (%.3f)" % (mean(n_scores), std(n_scores)))

# Show any plots (though no plot is explicitly created in this code)
pyplot.show()

We can see that the robust scaler transform results in a lift in
performance from 71.7 percent accuracy without the transform to about 73.4 percent with the
transform.

## Explore Robust Scaler Range

The range used to scale each variable is chosen by default as the IQR is bounded by the 25th
and 75th percentiles. This is specified by the quantile range argument as a tuple. Other
values can be specified and might improve the performance of the model, such as a wider range,
allowing fewer values to be considered outliers, or a more narrow range, allowing more values
to be considered outliers. The example below explores the effect of different definitions of the
range from 1st to the 99th percentiles to 30th to 70th percentiles.

In [ ]:
# explore the scaling range of the robust scaler transform

# get the dataset
def get_dataset():
    # Load the dataset from the specified CSV file without a header row
    dataset = read_csv(pima_indians_diabetes_csv, header=None)
    
    # Convert the dataset into a numpy array
    data = dataset.values
    
    # Separate the dataset into input features (X) and output labels (y)
    X, y = data[:, :-1], data[:, -1]  # Inputs (all columns except the last), Outputs (last column)
    
    # Ensure that the input features are of type float32 (for better performance)
    X = X.astype("float32")
    
    # Convert the output labels to integers using LabelEncoder (for classification tasks)
    y = LabelEncoder().fit_transform(y.astype("str"))  # Convert to string and then encode to integers
    
    # Return the prepared input and output data
    return X, y



# get a list of models to evaluate
def get_models():
    # Create an empty dictionary to store the models
    models = dict()
    
    # Iterate over a list of values to create different models
    for value in [1, 5, 10, 15, 20, 25, 30]:
        # Define the transformation step using RobustScaler with quantile_range based on 'value'
        trans = RobustScaler(quantile_range=(value, 100 - value))
        
        # Define the classifier model using KNeighborsClassifier
        model = KNeighborsClassifier()
        
        # Store the model in the dictionary, where the key is the string representation of 'value'
        # The pipeline includes both the transformation step and the model step
        models[str(value)] = Pipeline(steps=[("t", trans), ("m", model)])
    
    # Return the dictionary of models
    return models



# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    # Create a Repeated Stratified K-Fold cross-validation object
    # n_splits=10: 10-fold cross-validation (split data into 10 parts)
    # n_repeats=3: repeat the process 3 times to get more robust results
    # random_state=1: ensures reproducibility of results by fixing the random seed
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    
    # Perform cross-validation and calculate accuracy scores for each fold
    # scoring="accuracy" specifies that the evaluation metric is accuracy
    # n_jobs=-1: use all available CPUs to speed up the computation
    scores = cross_val_score(model, X, y, scoring="accuracy", cv=cv, n_jobs=-1)
    
    # Return the array of accuracy scores from each fold
    return scores

# Define the dataset by calling the function to load or generate it
X, y = get_dataset()

# Get the models to evaluate by calling a function that returns a dictionary of model names and their respective model objects
models = get_models()

# Initialize empty lists to store the evaluation results and model names
results, names = list(), list()

# Loop through each model and evaluate it
for name, model in models.items():
    # Evaluate the current model using the dataset X and y, and store the scores
    scores = evaluate_model(model, X, y)
    
    # Append the scores to the results list
    results.append(scores)
    
    # Append the model's name to the names list
    names.append(name)
    
    # Print the model's name along with its mean and standard deviation of the evaluation scores
    print(">%s %.3f (%.3f)" % (name, mean(scores), std(scores)))

We can see that ranges such as 10-90 and 15-85 perform better than the default
of 25-75.

In [ ]:
# Plotting a boxplot to compare the performance of different models

# 'results' contains the performance data for each model (e.g., accuracy scores, etc.)
# 'names' is a list of the model names corresponding to the performance results
# The 'boxplot' function is used to create the boxplot for comparing results visually
pyplot.boxplot(results, showmeans=True)  # Plot the boxplot with showing means

# Set the x-axis labels using 'xticklabels'
pyplot.xticks(ticks=range(1, len(names) + 1), labels=names)  # Set the model names on the x-axis

# Display the plot
pyplot.show()  # Show the generated plot to the user

Box and whisker plots are created to summarize the classification accuracy scores for each
IQR range. We can see a subtle difference in the distribution and mean accuracy with the larger
ranges of 15-85 vs 25-75. percentiles.

## Conclusion

Robust scaling provides an effective way to standardize numerical variables when outliers are present. Different scaling ranges (like 10-90 or 15-85 percentiles) can outperform the default 25-75 percentile range, showing the importance of testing different scaling parameters.

## Clean up

Remember to shut down your Jupyter Notebook environment and delete any unnecessary files or resources once you've completed the tutorial.